In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
Shotlog=pd.read_csv("C:/Users/trent/OneDrive/Documents/Repositories/NBA-HotHand-Analysis/data/Shotlog1.csv")
Player_Stats=pd.read_csv("C:/Users/trent/OneDrive/Documents/Repositories/NBA-HotHand-Analysis/data/Player_Stats1.csv")
Player_Shots=pd.read_csv("C:/Users/trent/OneDrive/Documents/Repositories/NBA-HotHand-Analysis/data/Player_Shots1.csv")
Player_Game=pd.read_csv("C:/Users/trent/OneDrive/Documents/Repositories/NBA-HotHand-Analysis/data/Player_Game1.csv")
Shotlog.head()

,team_previous_shot,player_position,home_game,location_x,opponent_previous_shot,home_team,shot_type,points,away_team,location_y,...,date,shoot_player,time_from_last_shot,quarter,current_shot_outcome,current_shot_hit,lag_shot_hit,average_hit,shot_count,shot_per_game
0,MISSED,PG,Yes,210.0,SCORED,ATL,Pullup Jump Shot,2,WAS,267.0,...,2016-10-27,Dennis Schroder,27.0,1,MISSED,0,1.0,0.451029,1215,12
1,MISSED,SF,Yes,279.0,SCORED,ATL,Jump Shot,3,WAS,130.0,...,2016-10-27,Kent Bazemore,4.0,1,MISSED,0,0.0,0.408587,722,7
2,MISSED,PG,Yes,58.0,SCORED,ATL,Driving Layup,2,WAS,262.0,...,2016-10-27,Dennis Schroder,50.0,1,MISSED,0,0.0,0.451029,1215,12
3,MISSED,C,Yes,107.0,SCORED,ATL,Turnaround Jump Shot,2,WAS,254.0,...,2016-10-27,Dwight Howard,3.0,1,MISSED,0,1.0,0.631922,614,9
4,MISSED,PF,Yes,167.0,MISSED,ATL,Pullup Jump Shot,2,WAS,306.0,...,2016-10-27,Paul Millsap,47.0,1,SCORED,1,0.0,0.442387,972,20


In [3]:
# Dummy variable for whether consecutive shots were made
Shotlog['conse_shot_hit'] = np.where((Shotlog['current_shot_hit']==1)&(Shotlog['lag_shot_hit']==1), 1, 0) 
Shotlog.head()

,team_previous_shot,player_position,home_game,location_x,opponent_previous_shot,home_team,shot_type,points,away_team,location_y,...,shoot_player,time_from_last_shot,quarter,current_shot_outcome,current_shot_hit,lag_shot_hit,average_hit,shot_count,shot_per_game,conse_shot_hit
0,MISSED,PG,Yes,210.0,SCORED,ATL,Pullup Jump Shot,2,WAS,267.0,...,Dennis Schroder,27.0,1,MISSED,0,1.0,0.451029,1215,12,0
1,MISSED,SF,Yes,279.0,SCORED,ATL,Jump Shot,3,WAS,130.0,...,Kent Bazemore,4.0,1,MISSED,0,0.0,0.408587,722,7,0
2,MISSED,PG,Yes,58.0,SCORED,ATL,Driving Layup,2,WAS,262.0,...,Dennis Schroder,50.0,1,MISSED,0,0.0,0.451029,1215,12,0
3,MISSED,C,Yes,107.0,SCORED,ATL,Turnaround Jump Shot,2,WAS,254.0,...,Dwight Howard,3.0,1,MISSED,0,1.0,0.631922,614,9,0
4,MISSED,PF,Yes,167.0,MISSED,ATL,Pullup Jump Shot,2,WAS,306.0,...,Paul Millsap,47.0,1,SCORED,1,0.0,0.442387,972,20,0


In [ ]:
# Dataframe for average probabilities of previous and consecutive shots made by each player
Player_Prob=Shotlog.groupby(['shoot_player'])[['conse_shot_hit','lag_shot_hit']].mean()
Player_Prob=Player_Prob.reset_index()
Player_Prob.rename(columns={'lag_shot_hit':'average_lag_hit'}, inplace=True)
Player_Prob.head()

,shoot_player,conse_shot_hit,average_lag_hit
0,A.J. Hammons,0.148148,0.407407
1,Aaron Brooks,0.184874,0.436975
2,Aaron Gordon,0.204082,0.460459
3,Adreian Payne,0.153846,0.435897
4,Al Horford,0.207367,0.469304


In [ ]:
# Calcualate conditional probability of making a shot given the previous shot
Player_Prob['conditional_prob']=Player_Prob['conse_shot_hit']/Player_Prob['average_lag_hit']
Player_Prob.head()

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob
0,A.J. Hammons,0.148148,0.407407,0.363636
1,Aaron Brooks,0.184874,0.436975,0.423077
2,Aaron Gordon,0.204082,0.460459,0.443213
3,Adreian Payne,0.153846,0.435897,0.352941
4,Al Horford,0.207367,0.469304,0.441860


In [ ]:
# Merge new variables into Player_Stats dataframe to see if conditional probability is better than unconditional probability
# Last 2 columns are conditional probabilities and unconditional probabilities
Player_Stats=pd.merge(Player_Prob, Player_Stats, on=['shoot_player'])
Player_Stats.head(10)

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob,average_hit
0,A.J. Hammons,0.148148,0.407407,0.363636,0.404762
1,Aaron Brooks,0.184874,0.436975,0.423077,0.403333
2,Aaron Gordon,0.204082,0.460459,0.443213,0.454861
3,Adreian Payne,0.153846,0.435897,0.352941,0.425926
4,Al Horford,0.207367,0.469304,0.441860,0.473159
5,Al Jefferson,0.226601,0.517241,0.438095,0.498938
6,Al-Farouq Aminu,0.170370,0.402469,0.423313,0.392704
7,Alan Anderson,0.134615,0.423077,0.318182,0.375000
8,Alan Williams,0.266667,0.524444,0.508475,0.516854
9,Alec Burks,0.159420,0.391304,0.407407,0.399194


In [8]:
Player_Stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   shoot_player      481 non-null    object 
 1   conse_shot_hit    481 non-null    float64
 2   average_lag_hit   481 non-null    float64
 3   conditional_prob  472 non-null    float64
 4   average_hit       481 non-null    float64
dtypes: float64(4), object(1)
memory usage: 18.9+ KB


In [9]:
# Drop values where conditional probability is NaN
Player_Stats=Player_Stats[pd.notnull(Player_Stats["conditional_prob"])]

In [ ]:
# Players with the hottest hands (highest conditional probability)
Player_Stats.sort_values(by=['conditional_prob'], ascending=[False]).head(10)

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob,average_hit
72,Chinanu Onuaku,0.333333,0.333333,1.000000,0.714286
293,Lucas Nogueira,0.495495,0.657658,0.753425,0.660256
36,Axel Toupane,0.500000,0.666667,0.750000,0.555556
343,Nick Collison,0.428571,0.571429,0.750000,0.608696
221,Joel Anthony,0.375000,0.500000,0.750000,0.625000
402,Salah Mejri,0.475610,0.658537,0.722222,0.642336
464,Udonis Haslem,0.357143,0.500000,0.714286,0.478261
102,DeAndre Jordan,0.493952,0.699597,0.706052,0.714038
398,Rudy Gobert,0.457565,0.653137,0.700565,0.662921
74,Chris McCullough,0.388889,0.555556,0.700000,0.500000


In [11]:
# Difference between conditional and unconditional probabilities
Player_Stats['diff_prob']=Player_Stats['conditional_prob']-Player_Stats['average_hit']
Player_Stats=pd.merge(Player_Stats, Player_Shots, on=['shoot_player'])
Player_Stats.sort_values(by=['diff_prob'], ascending=[False]).head(10)

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob,average_hit,diff_prob,shot_count
276,Lamar Patterson,0.090909,0.181818,0.500000,0.200000,0.300000,15
70,Chinanu Onuaku,0.333333,0.333333,1.000000,0.714286,0.285714,7
120,Diamond Stone,0.125000,0.250000,0.500000,0.230769,0.269231,13
455,Udonis Haslem,0.357143,0.500000,0.714286,0.478261,0.236025,23
215,Joe Young,0.217391,0.369565,0.588235,0.361111,0.227124,72
273,Kyle Wiltjer,0.166667,0.333333,0.500000,0.285714,0.214286,14
300,Marcus Georges-Hunt,0.250000,0.500000,0.500000,0.285714,0.214286,7
72,Chris McCullough,0.388889,0.555556,0.700000,0.500000,0.200000,32
35,Axel Toupane,0.500000,0.666667,0.750000,0.555556,0.194444,9
86,Damjan Rudez,0.230769,0.423077,0.545455,0.352273,0.193182,88


Performing T-test to see if conditional prob is significantly different than unconditional prob

In [12]:
import scipy.stats as sp

In [13]:
sp.stats.ttest_ind(Player_Stats['conditional_prob'], Player_Stats['average_hit'])

C:\Users\trent\AppData\Local\Temp\ipykernel_12328\1224862971.py:1: DeprecationWarning: Please import `ttest_ind` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  sp.stats.ttest_ind(Player_Stats['conditional_prob'], Player_Stats['average_hit'])


TtestResult(statistic=np.float64(-1.6259251871488678), pvalue=np.float64(0.10430003992138251), df=np.float64(942.0))

Autocorrelation Coefficient

In [14]:
# Autocorrelation Coefficient for current shot following previous shot
Shotlog['current_shot_hit'].corr(Shotlog['lag_shot_hit'])

np.float64(-0.0073232951280306)

In [15]:
# Autocorrelation Coefficient by player
Shotlog.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().head(10)

current_shot_hit  lag_shot_hit
shoot_player                                                  
A.J. Hammons  current_shot_hit          1.000000     -0.011562
              lag_shot_hit             -0.011562      1.000000
Aaron Brooks  current_shot_hit          1.000000      0.012709
              lag_shot_hit              0.012709      1.000000
Aaron Gordon  current_shot_hit          1.000000     -0.036678
              lag_shot_hit             -0.036678      1.000000
Adreian Payne current_shot_hit          1.000000     -0.236082
              lag_shot_hit             -0.236082      1.000000
Al Horford    current_shot_hit          1.000000     -0.056836
              lag_shot_hit             -0.056836      1.000000

In [16]:
# Unstack to make more readable
Autocorr_Hit=Shotlog.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().unstack()
Autocorr_Hit.head()

current_shot_hit                  lag_shot_hit             
              current_shot_hit lag_shot_hit current_shot_hit lag_shot_hit
shoot_player                                                             
A.J. Hammons               1.0    -0.011562        -0.011562          1.0
Aaron Brooks               1.0     0.012709         0.012709          1.0
Aaron Gordon               1.0    -0.036678        -0.036678          1.0
Adreian Payne              1.0    -0.236082        -0.236082          1.0
Al Horford                 1.0    -0.056836        -0.056836          1.0

In [17]:
# Select columns we want
Autocorr_Hit=Shotlog.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().unstack().iloc[:,1].reset_index()
Autocorr_Hit.head()

,shoot_player,current_shot_hit
,,lag_shot_hit
0,A.J. Hammons,-0.011562
1,Aaron Brooks,0.012709
2,Aaron Gordon,-0.036678
3,Adreian Payne,-0.236082
4,Al Horford,-0.056836


In [18]:
# Get rid of duplicate column names
Autocorr_Hit.columns=Autocorr_Hit.columns.get_level_values(0)
Autocorr_Hit.head()

,shoot_player,current_shot_hit
0,A.J. Hammons,-0.011562
1,Aaron Brooks,0.012709
2,Aaron Gordon,-0.036678
3,Adreian Payne,-0.236082
4,Al Horford,-0.056836


In [19]:
# Rename column to autocorr
Autocorr_Hit.rename(columns={'current_shot_hit':'autocorr'}, inplace=True)
Autocorr_Hit.head()

,shoot_player,autocorr
0,A.J. Hammons,-0.011562
1,Aaron Brooks,0.012709
2,Aaron Gordon,-0.036678
3,Adreian Payne,-0.236082
4,Al Horford,-0.056836


In [20]:
# Merge into Player_Stats to get shots per game
Player_Game_Shot=Player_Game.groupby(["shoot_player"])['shot_per_game'].mean().reset_index(name='avg_shot_game')
Player_Game_Shot.head()

,shoot_player,avg_shot_game
0,A.J. Hammons,2.80000
1,Aaron Brooks,4.83871
2,Aaron Gordon,10.80000
3,Aaron Harrison,1.00000
4,Adreian Payne,3.60000


In [21]:
Autocorr_Hit=pd.merge(Autocorr_Hit, Player_Game_Shot, on=['shoot_player'])
Autocorr_Hit.sort_values(by=['autocorr'], ascending=[False]).head(10)

,shoot_player,autocorr,avg_shot_game
279,Kyle Wiltjer,0.632456,1.750000
464,Udonis Haslem,0.428571,2.555556
330,Mike Miller,0.414758,2.300000
282,Lamar Patterson,0.388889,3.750000
122,Diamond Stone,0.333333,2.600000
229,Johnny O'Bryant III,0.301511,3.000000
36,Axel Toupane,0.250000,3.000000
202,Jarrod Uthoff,0.237826,4.750000
220,Joe Young,0.237004,2.769231
187,Jakob Poeltl,0.219336,2.613636


In [22]:
# Merge into Player_Shots to see number of shots taken
Player_Shots=pd.merge(Player_Shots, Player_Game_Shot, on=['shoot_player'])
Player_Shots.head()

,shoot_player,shot_count,avg_shot_game
0,A.J. Hammons,42,2.80000
1,Aaron Brooks,300,4.83871
2,Aaron Gordon,864,10.80000
3,Aaron Harrison,4,1.00000
4,Adreian Payne,54,3.60000


In [23]:
Shotlog.to_csv("C:/Users/trent/OneDrive/Documents/Repositories/NBA-HotHand-Analysis/data/Shotlog2.csv", index=False)
Player_Stats.to_csv("C:/Users/trent/OneDrive/Documents/Repositories/NBA-HotHand-Analysis/data/Player_Stats2.csv", index=False)
Player_Shots.to_csv("C:/Users/trent/OneDrive/Documents/Repositories/NBA-HotHand-Analysis/data/Player_Shots2.csv", index=False)